<a href="https://colab.research.google.com/github/Strata-Tech/Extracting-Lat-Long-from-OneMap/blob/main/extract_lat_long.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from datetime import datetime as dt
import seaborn as sns


In [30]:
files=['/content/drive/MyDrive/PCMLdata/resale-flat-prices-based-on-approval-date-1990-1999.csv','/content/drive/MyDrive/PCMLdata/resale-flat-prices-based-on-approval-date-2000-feb-2012.csv','/content/drive/MyDrive/PCMLdata/resale-flat-prices-based-on-registration-date-from-jan-2015-to-dec-2016.csv','/content/drive/MyDrive/PCMLdata/resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv','/content/drive/MyDrive/PCMLdata/resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv']

In [31]:
for x in files:
    df=pd.read_csv(x)
    if x==files[0]:
        dataset=df
        
    else:
        dataset=pd.concat([dataset,df])

In [32]:
dataset.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0,NaN
1,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,IMPROVED,1977,6000.0,NaN
2,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,8000.0,NaN
3,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,IMPROVED,1977,6000.0,NaN
4,1990-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,NEW GENERATION,1976,47200.0,NaN


In [33]:
import requests


In [34]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 826581 entries, 0 to 52202
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   month                826581 non-null  object 
 1   town                 826581 non-null  object 
 2   flat_type            826581 non-null  object 
 3   block                826581 non-null  object 
 4   street_name          826581 non-null  object 
 5   storey_range         826581 non-null  object 
 6   floor_area_sqm       826581 non-null  float64
 7   flat_model           826581 non-null  object 
 8   lease_commence_date  826581 non-null  int64  
 9   resale_price         826581 non-null  float64
 10  remaining_lease      117527 non-null  object 
dtypes: float64(2), int64(1), object(8)
memory usage: 75.7+ MB


In [38]:
dataset['address']=dataset['block']+" "+dataset['street_name']
addresslist=list(dataset['address'])

Defining a function as getcoordinates . Using the requests module, I will use it to search information on OneMap API. The address will have to be inserted as a string into the the given URL in order to retrieve information. Next, is to use the eval() function to evaluate it as a Python expression. As not all addresses will have coordinates, I use an if-else clause to only take those with results.

In [36]:
def getcoordinates(address):
    req = requests.get('https://developers.onemap.sg/commonapi/search?searchVal='+address+'&returnGeom=Y&getAddrDetails=Y&pageNum=1')
    resultsdict = eval(req.text)
    if len(resultsdict['results'])>0:
        return resultsdict['results'][0]['LATITUDE'], resultsdict['results'][0]['LONGITUDE']
    else:
        pass

In [ ]:
coordinateslist= []
count = 0
failed_count = 0
for address in addresslist:
    try:
        if len(getcoordinates(address))>0:
            count = count + 1
            print('Extracting',count,'out of',len(addresslist),'addresses')
            coordinateslist.append(getcoordinates(address))
    except:
        count = count + 1           
        failed_count = failed_count + 1
        print('Failed to extract',count,'out of',len(addresslist),'addresses')
        coordinateslist.append(None)
print('Total Number of Addresses With No Coordinates',failed_count)

Failed to extract 1 out of 826581 addresses
Failed to extract 2 out of 826581 addresses
Failed to extract 3 out of 826581 addresses
Failed to extract 4 out of 826581 addresses
Extracting 5 out of 826581 addresses
Extracting 6 out of 826581 addresses
Extracting 7 out of 826581 addresses
Extracting 8 out of 826581 addresses
Extracting 9 out of 826581 addresses
Extracting 10 out of 826581 addresses
Extracting 11 out of 826581 addresses
Extracting 12 out of 826581 addresses
Extracting 13 out of 826581 addresses
Extracting 14 out of 826581 addresses
Extracting 15 out of 826581 addresses
Failed to extract 16 out of 826581 addresses
Extracting 17 out of 826581 addresses
Failed to extract 18 out of 826581 addresses
Extracting 19 out of 826581 addresses
Extracting 20 out of 826581 addresses
Extracting 21 out of 826581 addresses
Extracting 22 out of 826581 addresses
Extracting 23 out of 826581 addresses
Extracting 24 out of 826581 addresses
Extracting 25 out of 826581 addresses
Extracting 26 out

Firstly to create an empty list to append the extracted coordinates. Next, is to put a count function so that we will know where the progress is currently at and the total number of addresses that doesn’t have coordinates. 

This allows to keep track. A try else condition which will allow the for loop to run continuously even if one of the addresses fail to retrieve any coordinates from the API. 

This is particularly useful to ensure the loop runs smoothly, especially for dirty datasets that are not able to generate any results from the API. By doing so,we are still able to attain the bulk of coordinates. After which, we can  then filter for the remaining dirty addresses and clean those easily.

Combine the datasets

In [ ]:

df_coordinates = pd.DataFrame(coordinateslist)
df_combined = dataset.join(df_coordinates)
# df_combined  = df_combined.rename(columns={0:'Latitude', 1:'Longitude'}
  
  
  